<a href="https://colab.research.google.com/github/abadabada/MachineLearningMiniProject/blob/main/MachineLearningMiniProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Machine Learning Mini-Project: Pixel-Aligned Implicit Function for High-Resolution 3D Human Digitization (PIFuHD)**

In this project, I am working with the Multi-Level Pixel-Aligned Implicit Function for
High-Resolution 3D Human Digitization (PIFuHD). This ML-based application takes a PNG or JPEG of a clothed human as an input, then produces a 3D model based on the picture as an OBJ file.


This model uses Deep Learning and more specifically convolutional neural networks (CNN) to create the 3d objects. This technique is perfect for this model since it uses Images as an input and doesn't produce any 3D data during processing whichc makes it very memory efficient. Althought since this is still a CNN I chose to run in google colab due to it's heavy requirment of GPU. This model first uses a Lightweight Human Pose Estimation, then the image is cropped to a 512x512 rect version. After this the result picture is used as an input in the PIFu model. The PIFu model then also creates a height map picture of the back and the front to help with determining the positon of the 3D points in space for a higher resolution.


Since I am using Deep Learning, extraction techniques aren't required.


I'm also using a premade large scale data set provided by the creators of the model. This data set is generated by scanning numorous high quality 3d human meshes and it is used to train the model in an end-to-end fashion.


To evalutate this system, I will be using different picture types than just front portaits of humans. I still will be keep it into the humanoid category, by using either CG characters, like aliens or 3d characters from movies or games, and also testing it with drawn characters, and also mix in using different shapes and angles of these characters to see how well the model will handle such input.

My only original contribution to this project is choosing the input and testing with it various times to see if there will be any variety in the output from the same image, else the code is mostly from the PIFu Github repository and the data set is also provided with that.

**Mounting Google Drive to Google Colab Notebook**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**PIFuHD Repository**



In [106]:
!git clone https://github.com/facebookresearch/pifuhd

fatal: destination path 'pifuhd' already exists and is not an empty directory.


**Input Data & Output Path**

In [140]:
cd /content/pifuhd/sample_images

/content/pifuhd/sample_images


In [141]:
import os

image_path = '/content/pifuhd/sample_images/Shrek.png' # example image
image_dir = os.path.dirname(image_path)
file_name = os.path.splitext(os.path.basename(image_path))[0]

#output path
obj_path = '/content/pifuhd/results/pifuhd_final/recon/result_%s_256.obj' % file_name
out_img_path = '/content/pifuhd/results/pifuhd_final/recon/result_%s_256.png' % file_name

In [142]:
cd /content

/content


**Image Cripping & Human Pose Estimation Training Model**

In [143]:
!git clone https://github.com/Daniil-Osokin/lightweight-human-pose-estimation.pytorch.git

fatal: destination path 'lightweight-human-pose-estimation.pytorch' already exists and is not an empty directory.


In [144]:
cd /content/lightweight-human-pose-estimation.pytorch/

/content/lightweight-human-pose-estimation.pytorch


In [145]:
!wget https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth

--2023-02-13 09:01:11--  https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth
Resolving download.01.org (download.01.org)... 104.87.100.39, 2600:1413:b000:38e::4b21, 2600:1413:b000:389::4b21
Connecting to download.01.org (download.01.org)|104.87.100.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87959810 (84M) [application/octet-stream]
Saving to: ‘checkpoint_iter_370000.pth.12’

checkpoint_iter_370 100%[===================>]  83.88M   301MB/s    in 0.3s    

2023-02-13 09:01:12 (301 MB/s) - ‘checkpoint_iter_370000.pth.12’ saved [87959810/87959810]



In [146]:
import torch
import cv2
import numpy as np
from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
import demo

def get_rect(net, images, height_size):
    net = net.eval()

    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    previous_poses = []
    delay = 33
    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        orig_img = img.copy()
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []

        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            valid_keypoints = []
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])
            valid_keypoints = np.array(valid_keypoints)
            
            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
              pmin = valid_keypoints.min(0)
              pmax = valid_keypoints.max(0)

              center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int)
              radius = int(0.65 * max(pmax[0]-pmin[0], pmax[1]-pmin[1]))
            elif pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:
              # if leg is missing, use pelvis to get cropping
              center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.int)
              radius = int(1.45*np.sqrt(((center[None,:] - valid_keypoints)**2).sum(1)).max(0))
              center[1] += int(0.05*radius)
            else:
              center = np.array([img.shape[1]//2,img.shape[0]//2])
              radius = max(img.shape[1]//2,img.shape[0]//2)

            x1 = center[0] - radius
            y1 = center[1] - radius

            rects.append([x1, y1, 2*radius, 2*radius])

        np.savetxt(rect_path, np.array(rects), fmt='%d')

In [147]:
net = PoseEstimationWithMobileNet()
checkpoint = torch.load('checkpoint_iter_370000.pth', map_location='cpu')
load_state(net, checkpoint)

get_rect(net.cuda(), [image_path], 512)

<ipython-input-146-1919f42bf24c>:53: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int)


**Pretrained Model**

In [148]:
cd /content/pifuhd/

/content/pifuhd


In [149]:
!sh ./scripts/download_trained_model.sh

+ mkdir -p checkpoints
+ cd checkpoints
+ wget https://dl.fbaipublicfiles.com/pifuhd/checkpoints/pifuhd.pt pifuhd.pt
--2023-02-13 09:01:12--  https://dl.fbaipublicfiles.com/pifuhd/checkpoints/pifuhd.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1548375177 (1.4G) [application/octet-stream]
Saving to: ‘pifuhd.pt.2’

pifuhd.pt.2         100%[===================>]   1.44G  16.5MB/s    in 93s     

2023-02-13 09:02:47 (15.8 MB/s) - ‘pifuhd.pt.2’ saved [1548375177/1548375177]

--2023-02-13 09:02:47--  http://pifuhd.pt/
Resolving pifuhd.pt (pifuhd.pt)... failed: Name or service not known.
wget: unable to resolve host address ‘pifuhd.pt’
FINISHED --2023-02-13 09:02:47--
Total wall clock time: 1m 35s
Downloaded: 1 files, 1.4G in 1m 33s (15.8 MB/s)


**Running PIHuHD & Output Data**

In [150]:
!python -m apps.simple_test -r 256 --use_rect -i $image_dir

Resuming from  ./checkpoints/pifuhd.pt
test data size:  4
initialize network with normal
initialize network with normal
generate mesh (test) ...
  0% 0/4 [00:00<?, ?it/s]./results/pifuhd_final/recon/result_JohnCena_256.obj
/content/pifuhd/lib/mesh_util.py:77: FutureWarning: marching_cubes_lewiner is deprecated in favor of marching_cubes. marching_cubes_lewiner will be removed in version 0.19
  verts, faces, normals, values = measure.marching_cubes_lewiner(sdf, thresh)
 25% 1/4 [00:10<00:31, 10.60s/it]./results/pifuhd_final/recon/result_Shrek_256.obj
 50% 2/4 [00:19<00:19,  9.67s/it]./results/pifuhd_final/recon/result_alien_256.obj
 75% 3/4 [00:25<00:07,  7.99s/it]./results/pifuhd_final/recon/result_test_256.obj
100% 4/4 [00:31<00:00,  7.82s/it]
